In [1]:
#FILE CON I MIEI TENTATIVI DI PREPROCESSING SUL DATABASE DI ANOBii
from pyspark.sql.functions import lower
from pyspark.sql.types import IntegerType, StringType
file = "/data/SMARTDATA/books/anobii/author_item.csv"
DFauthorbooks = spark.read.csv(file,header=True) #MOSTRA I LIBRI RELATIVI AGLI AUTORI
#DFauthorbooks.show(1, False)

#LIBRI DEGLI AUTORI

In [2]:
file = "/data/SMARTDATA/books/anobii/author_display.csv"
DFdisplay = spark.read.csv(file,header=True) #MOSTRA GLI AUTORI DEL DATABASE
#DFdisplay.show()
#DFauthorbooks_withnames = DFauthorbooks.join(DFdisplay, DFauthorbooks.author_id == DFdisplay.author_id).drop(DFdisplay.author_id)
#AUTORI

In [3]:
#A quanto pare author_display è una tabella contenente tutti gli autori del database di anobii mentre author_items
#contiene i diversi libri associati ad ogni autore. 
#DUBBIO: cosa è author original? Cerchiamo una tupla in cui author_original NON è NULL:

file = "/data/SMARTDATA/books/anobii/item.csv"
DFitems_anobii = spark.read.csv(file,header=True) #MOSTRA I LIBRI DEL DATABASE
#DFitems.show()

#DFitemsauthor = DFitems_anobii.join(DFauthorbooks_withnames, DFitems_anobii.item_id == DFauthorbooks.item_id).drop(DFitems_anobii.item_id)
#DFitemsauthor.filter(DFitemsauthor.author_id_original != "null").show(20, False)
#DFitemsauthor.show(1, False)
#AUTHOR: JOHN FOWLES
#DFdisplay.filter(DFdisplay.author_id == "360704").show(20, False)
#DFdisplay.filter(DFdisplay.author_id_original == "72565").show(20, False)

#IPOTESI: Sembra che author_id_original sia semplicemente l'id di un vecchio database non a nostra disposizione
#dato che non è un autore (author_id) presente in questo DB

#Possiamo anche notare come language sia il linguaggio del libro mentre language_correct sia il linguaggio del
#libro originale (notare la tupla item_id 1002883)

In [4]:
file = "/data/SMARTDATA/books/anobii/language_mapping.csv"
DFlanguages = spark.read.csv(file,header=True)
#DFlanguages.show()

#Dobbiamo tenere solo i libri con language=11 #FILTRO LINGUAGGIO



In [5]:
file = "/data/SMARTDATA/books/anobii/item_recommend.csv"
DFrecommends = spark.read.csv(file,header=True)
#DFrecommends.show(50, False)

#Questa tabella contiene gli ISBN riferiti all'item Id (quindi item_id non è unico?)

In [6]:
#Per coerenza, procediamo a un filtraggio equivalente a quello fatto per le biblioteche:
#FILTRAGGI BASE

#FILTRAGGIO 1: solo libri italiani (filtrare language==11)
#FILTRAGGIO 2: solo monografie e manoscritti (questo a dire il vero è fattibile in maniera un po' diversa dal caso
#delle biblioteche poiché l'attributo "binding" indica la rilegatura del libro). Per iniziare, considererò
#"paperback" (copertina flessibile) e "hardcover" (copertina rigida) NOTA: probabilmente ulteriori scremature
#da questo punto di vista saranno possibili
#FILTRAGGIO 3: eliminare fumetti e magazine (non sono sicuro della loro presenza, controllo)
#UPDATE: non solo ci sono ma sono pure tantissimi! Filtraggio doveroso

#DFitems.filter(DFitems.title.contains("Topolino")).show(20, False)

#FILTRAGGI AVANZATI

#FILTRAGGIO 4: Libri con più di 50 voti (Dove prendere i voti?) In item_comment_vote ci sono i commenti ai libri
#fatti da vari utenti. Tuttavia, non vedo i voti. item_comment_feedback contiene i commenti veri e propri.
#In item_comment_vote sembrano esserci due campi "total_yes" e "total_no": che siano loro i voti? (Consigliato
#Non consigliato). UPDATE: Noto che in item_comment_vote è presente l'average rating: è già qualcosa, tuttavia
#vorrei trovare i voti dei singoli utenti.

#FILTRAGGIO 5: Pulizie eventuali, su anobii i titoli sembrano già puliti quindi è necessario semplicemente mappare
#l'id dell'autore con il nome dell'autore.

file = "/data/SMARTDATA/books/anobii/item_comment_vote.csv" #MOSTRA INFORMAZIONI SUL VOTO DEGLI UTENTI
DFvotes = spark.read.csv(file,header=True)
#DFvotes.show(50, False)

file = "/data/SMARTDATA/books/anobii/item_comment_feedback.csv" #MOSTRA IL VOTO MEDIO
DFfeedback = spark.read.csv(file,header=True)
#DFfeedback.show(50, False)

file = "/data/SMARTDATA/books/anobii/person_item_recommend.csv" #MOSTRA I COMMENTI DEGLI UTENTI RELATIVI A UN LIBRO
DFcomments = spark.read.csv(file,header=True)
#DFcomments.show(50, False)

file = "/data/SMARTDATA/books/anobii/person_item_progress.csv" #MOSTRA INFORMAZIONI SULLE RACCOMANDAZIONI
DFprogress = spark.read.csv(file,header=True)
#DFprogress.show(50, False)

file = "/data/SMARTDATA/books/anobii/link_person_item_comment.csv" #MOSTRA INFORMAZIONI VARIE QUALI LA PRESENZA DI
#SPOILER IN UN COMMENTO O LA PRESENZA DI PROFANITà
DFinfo = spark.read.csv(file,header=True)
#DFinfo.show(50, False)

In [7]:
#INIZIAMO A FILTRARE USANDO LA ROADMAP DELLA CELLA POCO SOPRA

#FILTRAGGIO 1: linguaggio italiano
#Mi basterà filtrare da item.csv tutte le tuple con language diverso da 11.

DFfilteredlanguageitems = DFitems_anobii.filter(DFitems_anobii.language == "11")
#DFfilteredlanguageitems.show(20, False)

#FILTRAGGIO 2: solo "hardcover" e "paperback"

DFfilteredbindingitems = DFfilteredlanguageitems.filter((DFfilteredlanguageitems.binding == "Paperback") | (DFfilteredlanguageitems.binding == "Hardcover"))
#DFfilteredbindingitems.show(20, False)

#FILTRAGGIO 3: eliminare i fumetti (per quanto possibile)
#Userò come parole chiave personaggi famosi Disney quali Topolino e Paperino e i fumetti Bonelli (che fra gli italiani
#sono probabilmente i più gettonati)

DFfilteredmagazineitems = DFfilteredbindingitems.filter(~(DFfilteredbindingitems.title.contains("Topolino")) | (DFfilteredbindingitems.title.contains("Paperino"))|(DFfilteredbindingitems.title.contains("Tex"))|(DFfilteredbindingitems.title.contains("Dylan Dog"))|(DFfilteredbindingitems.title.contains("Nathan Never"))|(DFfilteredbindingitems.title.contains("Zagor")))
#DFfilteredmagazineitems.show(20, False)

#FILTRAGGIO 4: libri con più di X votes. Per vedere i voti di ogni libro dovrei usare il dataframe link_person_item.csv.
#Come possiamo notare, alcune recensione hanno valore 0: questo non vuol dire che il libro è stato valutato
#atrocemente ma che l'utente ha letto il libro senza inserire voti.

#Dobbiamo dunque scremare tali voti da questo dataframe.

file = "/data/SMARTDATA/books/anobii/link_person_item.csv" #MOSTRA IL VOTO DI UN UTENTE A UN LIBRO
DFstars = spark.read.csv(file,header=True)
#DFstars.show(50, False)

DFstarsfilteredno0 = DFstars.filter(DFstars.item_review > 0)
#DFstarsfilteredno0.show(20,False)

#Contiamo i libri con più voti e visualizziamoli in ordine discendente

DFgrouped = DFstarsfilteredno0.groupby("item_id").count().withColumnRenamed("count","total_votes").sort("total_votes", ascending=False)
#I voti dei libri più votati si aggirano intorno ai 10^4 come ordine di grandezza per cui per ora scelgo 300 come
#soglia di voti necessaria a rimanere nel dataframe (soggetto a cambiamenti)
DFgroupedfiltered = DFgrouped.filter(DFgrouped['total_votes'] > 300)
DFjoinbookstars = DFfilteredmagazineitems.join(DFgroupedfiltered, DFfilteredmagazineitems.item_id == DFgroupedfiltered.item_id).drop(DFgroupedfiltered.item_id)

#FILTRAGGIO 5: Pulizie varie ed eventuali
#DFfilteredlessthan10items.filter(DFfilteredlessthan10items.total_review > 100).show()

#NOTA: prendo i titoli con le minuscole per facilità di merging con le biblioteche
DFfiltered = DFjoinbookstars.select("item_id", lower(DFjoinbookstars.title), lower(DFjoinbookstars.sub_title), "isbn", "average_rating", "total_review", "total_wishlist", DFjoinbookstars['total_votes']).withColumnRenamed("lower(title)", "title").withColumnRenamed("lower(sub_title)", "sub_title")
#DFfiltered.sort("total_votes", ascending=False).show(20, False)
#DFfiltered.count() #2516 books (maybe too few? Let's try with 300 WITH 300: 4485, seems fair)

In [8]:
#Let's join votes and books to get titles (not considering votes with 0)

DFvoteswithtitles = DFstarsfilteredno0.join(DFfiltered, DFfiltered.item_id == DFstarsfilteredno0.item_id).select(DFstarsfilteredno0.item_id, "person_id", "title", str("sub_title"), "item_review", "total_review", "average_rating", "total_votes", "isbn")
#DFvoteswithtitles.filter((DFvoteswithtitles.title == "il fuggiasco") & (DFvoteswithtitles.person_id == "262165")).show()
#DFvoteswithtitles.filter(DFvoteswithtitles.title == "la custode di libri").show()

In [9]:
#from lightfm import LightFM
from pyspark.sql.functions import row_number, lit, dense_rank
from pyspark.sql import functions as F, Window
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
#Per ora proverò a costruire una CSR (matrice sparsa compressa) dal dataset di anobii (senza join con le biblioteche,
#solo per testare). La CSR è costruita in modo tale da avere tre vettori numpy, uno per i dati e due per gli indici
#riga-colonna con cui poi ricostruire la matrice originale.

#Come posso passare dal dataset a questa matrice? La maniera più efficiente (ed è meglio adottarle dato le grandezze
#dei dataset) è quella di creare la matrice direttamente, senza prima crearne una vuota da riempire.

#Supposto che le righe di tale matrice siano gli utenti e le colonne siano i libri (e i dati siano per ora solo
#i voti, poi potremo aggiungere feature per i metadati) allora posso provare ad aggiungere al mio dataframe delle 
#colonne indicanti gli indici futuri di riga colonna, usando degli id incrementali.

#Proviamo. (Forse con gli RDD è meglio)
RDDvoteswithtitles = DFvoteswithtitles.rdd.cache()
#RDDpair = RDDvoteswithtitles.map(lambda x: (x.person_id, list(x))).sortByKey()#.collect()

def create_user_dictionary(rdd): #Used to assign an integer id to each user of the rdd (to get the rows of the CSR)
    rdd = rdd.map(lambda x: (str(x.person_id), list(x))).sortByKey()
    user_dictionary = rdd.countByKey()
    i = 0
    for key in user_dictionary.keys():
        user_dictionary[key] = i
        i += 1
    return user_dictionary

def create_book_dictionary(rdd): #Used to assign an integer id to each book of the rdd (to get the column of the CSR)
    rdd = rdd.map(lambda x: (x.book_id, list(x))).sortByKey()
    book_dictionary = rdd.countByKey()
    i = 0
    for key in book_dictionary.keys():
        book_dictionary[key] = i
        i += 1
    return book_dictionary

#user_dictionary = create_user_dictionary(RDDvoteswithtitles)
#book_dictionary = create_book_dictionary(RDDvoteswithtitles)

def addDataType1(line):
    return Row(line.item_id, line.person_id, line.title, str(line.sub_title), line.item_review, line.total_review, line.average_rating, str(line.total_votes), line.isbn, "anobii")
RDDmapped = RDDvoteswithtitles.map(addDataType1)

#print(RDDmapped.take(20))

DFwithindexes = RDDmapped.toDF(["item_id", "person_id", "title", "sub_title",  "item_review", "total_review", "average_rating", "total_votes", "isbn", "data_type"], sampleRatio = 0.9) #Per evitare il problema del sampling in sub_title
#DFwithindexes.show(20, False)
#DFvoteswithtitles.filter((DFvoteswithtitles.title == "il fuggiasco") & (DFvoteswithtitles.person_id == "262165")).show()
#DFwithindexes contiene i voti di ogni utente dato il libro, altre info utili e gli indici di riga e di colonna
#della eventuale compressed sparse matrix per una facile ed efficiente creazione (per i sistemi di raccomandazione)


In [10]:
#Fatto ciò, riprendiamo il database delle biblioteche filtrato (come si importa un altro file .ipynb?)

%run cleanedData.ipynb #Così a quanto pare

Togli oggetti inutili: Prima 290125  - Dopo 257054
Togli libri in lingua straniera: Prima 290125  - Dopo 228059
Togli Fumetti: Prima 290125  - Dopo 227662
prefiltrati: 210633 - filtrati: 17702
prefiltrati: 210633 - filtrati: 17929
prefiltrati: 998403 - filtrati: 215446
prefiltrati: 5484078 - filtrati: 2154533
i libri sono: 17929


In [11]:
#Per adesso, concentriamoci su DFloand_definitive (i prestiti scremati).
DFauthorsandbooks = DFdisplay.join(DFauthorbooks, DFdisplay.author_id == DFauthorbooks.author_id).drop(DFdisplay.author_id).filter(DFdisplay.language=="11")
#DFauthorsandbooks.filter(DFauthorsandbooks.item_id == "1254828").show()
#DFauthorsandbooks.filter(DFauthorsandbooks.item_id=="651589").show(20, False)
DFwithindexes_and_authors = DFwithindexes.join(DFauthorsandbooks, DFwithindexes.item_id == DFauthorsandbooks.item_id).select(DFwithindexes.item_id, "person_id", "title", "sub_title", "author_name", "item_review", "total_review", "average_rating", "total_votes", "isbn", "data_type")
#Prendo i prestiti (che alla fine, è ciò che mi interessa) con i titoli che, per una formattazione più agevole, rendo minuscoli.
DFloans_with_titles = DFloans_definitive.join(DFmanifestations_definitive, DFloans_definitive.manifestation_id_new == DFmanifestations_definitive.manifestation_id_new).select(DFloans_definitive.manifestation_id_new, "patron_id_md5", "author", lower(DFmanifestations_definitive.title)).withColumnRenamed("lower(title)", "title")

#DFwithindexes_and_authors.filter((DFwithindexes_and_authors.title == "il fuggiasco") & (DFwithindexes_and_authors.person_id == "262165")).show()
#Di cosa ho bisogno in loans_definitive per renderlo "simile" alle tuple di anobii? Certamente avrò bisogno del titolo, poi del manifestation_id (item_id), del cliente della biblioteca
#(orrispettivo di person_id), di assegnare una item_review provvisoria (abbiamo stabilito, almeno per ora, 4 NOTA: l'average rating lo setto a NULL nelle tuple o lo ricalcolo considerando le 
#biblioteche? Da vedere), di settare le total_review (per ora a NULL?), i total_votes, l'isbn e lo user e book index (a NULL?)

#Il problema di molti campi che lascerei a NULL è che in alcuni casi i titoli non sono esattamente gli stessi a causa della formattazione dei database delle biblioteche. Per stemperare il problema,
#proverò a scindere titolo e sottotitoli in due campi differenti (alla maniera di anobii) e a pulire eventuali simboli strani (ad esempio, i numeri all'inizio di alcuni titoli).

RDDloans_with_titles = DFloans_with_titles.rdd

def title_and_subtitle(line): #Usata per splittare titolo e sottotitolo nel database delle biblioteche e renderli più simili ad anobii
    if len(line.title.split(" : ")) > 1: #Il libro possiede un sottotitolo
        title = line.title.split(" : ")[0]
        sub_title = line.title.split(" : ")[1]
        
        return Row(line.manifestation_id_new, line.patron_id_md5, title, sub_title, line.author, 4, "None", "None", "None", "None","biblioteche") #NE APPROFITTO ANCHE PER INSERIRE VOTO E CAMPI PER IL MERGE (PER ORA NULL)
    else: #Il libro non possiede un sottotitolo
        return Row(line.manifestation_id_new, line.patron_id_md5, line.title, "None", line.author, 4, "None", "None", "None", "None", "biblioteche")
    
RDDloans_for_merge = RDDloans_with_titles.map(title_and_subtitle)  

DFloans_to_merge = RDDloans_for_merge.toDF(["item_id", "person_id", "title", "author_name", "sub_title", "item_review", "total_review", "average_rating", "total_votes", "isbn", "data_type"])

from pyspark.sql.functions import rand

DFmerged = DFwithindexes_and_authors.union(DFloans_to_merge)



In [12]:
### MERGE AVVENUTO (sperando di averlo implementato bene...)

#Adesso vorrei cercare di sostituire i book_id delle biblioteche con quelli del corrispettivo titolo di anobii. Ciò aiuterà quando creerò la colonna degli indici per la compressed sparse matrix.
#Inoltre, vorrei che la funzione aggiornasse le metriche delle tuple delle biblioteche, per ora cosparse di "None".

#Come discusso nel meeting, vorrei inoltre eliminare dal mio dataframe mergiato tutti quei titoli con una sola parola (che di solito si riferiscono a guide turistiche).

#Infine, vorrei inserire gli indici di colonna e di riga della compressed sparse matrix in modo da iniziare a preparare l'input per la libreria di raccomandazione (ho già implementato le funzioni
#per farlo, a patto che le chiavi usate per il dizionario non siano erronee).

#FASE 1: i libri con lo stesso titolo dovrebbero avere lo stesso book_id (e le metriche dovrebbero essere aggiornate di conseguenza).
#Ciò è fattibile con la window di SQL e una partitionby. Controlliamo, con un sampling casuale, se ci sono due libri provenienti dai due diversi database (controllare data_type).

#DFmerged.filter(DFmerged.title == "la custode di libri").show() #TROVATO ERRORE DI RIPETIZIONE TUPLE A CAUSA DELLA MANCATA SCREMATURA DEL LINGUAGGIO DURANTE IL JOIN PER LE INFO SULL'AUTORE (SISTEMATO)

#ERRORE 2: alcuni libri di anobii sono ripetuti due volte con un autore diverso (ma perché?), da sistemare con un'aggregazione
#Prima, però. filtriamo via i titoli con una parola (dovrebbero essere pochi rispetto alla cardinalità totale e ci semplifica la vita con i libri omonimi)

DFmerged_no_onewordtitles = DFmerged.filter(DFmerged.title.contains(" ")) #Se contiene uno spazio allora non è una sola parola (sperando di aver formattato bene)
#DFmerged_no_onewordtitles.filter(DFmerged_no_onewordtitles.title == "barcellona").show()
#DFmerged.filter(DFmerged.title == "barcellona").show()

#Procediamo adesso con la sostituzione dei book_id delle biblioteche con quelli di anobii
DFmerged_aggregated = DFmerged_no_onewordtitles.select("*", F.min(DFmerged_no_onewordtitles.item_id).over(Window.partitionBy(DFmerged_no_onewordtitles.title)).alias("book_id"))

#Aggreghiamo adesso ripetizioni di uno stessa coppia (item_id, user_id) (PER ELIMINARE LE TUPLE RIPETUTE, VEDESI ERRORE 2 DOVUTO ALLA FORMATTAZIONE DI ANOBII)
#DFmerged_aggregated.filter(DFmerged_aggregated.title == "la custode di libri").show()

#Troviamo "new_author" formattato in maniera coerente fra i database mergiati
DFmerged_aggregated_norepetition = DFmerged_aggregated.select("*", F.min(DFmerged_aggregated.author_name).over(Window.partitionBy(DFmerged_aggregated.book_id)).alias("new_author"))

#DFmerged_aggregated_norepetition.filter(DFmerged_aggregated_norepetition.title == "la custode di libri").show()

#Infine, eliminiamo le ridondanze con dropDuplicates.

DFmerged_aggregated_norepetition_unique = DFmerged_aggregated_norepetition.dropDuplicates(['book_id', 'person_id'])
#DFmerged_aggregated_norepetition_unique.filter(DFmerged_aggregated_norepetition_unique.title == "la custode di libri").show()

#Infine, droppiamo gli attributi che non servono più e rinominiamo quelli nuovi e più utili
DFmerged_final = DFmerged_aggregated_norepetition_unique.drop("item_id").drop("author_name")
#ERRORE 2 SISTEMATO
#DFmerged_final.filter(DFmerged_final.book_id == "1254828").show()

In [ ]:
DFmerged_final.orderBy(rand()).show(50, False)

In [13]:
#Adesso che il merge e gli attributi sono un po' più solidi possiamo provare a calcolare qualche metrica anche per le tuple della biblioteca
#Prima però vorrei controllare una cosa

#DFmerged_final.filter(DFmerged_final.title == "1 ora").show()
#DFmerged_final.filter(DFmerged_final.title == "un'ora").show()

#Ho provato superficialmente a considerare libri con un numero nel titolo per controllare se fossero formattati diversamente, continuerò il controllo più tardi
#Una metrica da considerare per le tuple "biblioteche" potrebbe essere il numero di prestiti per un certo libro (sapere che un prestito è avvenuto 20 o 2000 volte è certamente un'info notevole)

#Nota: sarebbe consono anche aggiornare le metriche di anobii post raggruppamento
DFmerged_count = DFmerged_final.select("*", F.count("*").over(Window.partitionBy(DFmerged_final.book_id, DFmerged_final.data_type)).alias("total_count"))
#DFmerged_count.orderBy(rand()).show(50, False)
DFmerged_average = DFmerged_count.select("*", F.avg(DFmerged_final.item_review).over(Window.partitionBy(DFmerged_final.book_id, DFmerged_final.data_type)).alias("new_average_rating"))
DFmerged_maxreviews = DFmerged_average.select("*", F.max(DFmerged_final.total_review).over(Window.partitionBy(DFmerged_final.book_id, DFmerged_final.data_type)).alias("new_total_review"))

#NOTA: l'average rating per le tuple delle biblioteche sarà, almeno per ora, ovviamente 4.0 per via dei voti fissi. new_number_reviews rappresenta il numero massimo di recensioni in caso di incongruenze.
#DFmerged_sumreviews.orderBy(rand()).show(50, False)

#Infine, inseriamo l'isbn anche per le tuple delle biblioteche  che abbiano una corrispondenza in anobii(potrebbe servire ed è anche utile per ricercare eventuali libri dalla formattazione errata)
DFmerged_isbn = DFmerged_maxreviews.select("*", F.min(DFmerged_maxreviews.isbn).over(Window.partitionBy(DFmerged_final.book_id)).alias("new_isbn"))
#DFmerged_isbn.orderBy(rand()).show(20, False)  

#DFmerged_isbn.orderBy(rand()).filter(DFmerged_isbn.title == "i giorni dell'arcobaleno").show(50) #LIBRO PRESENTE SOLO NELLE BIBLIOTECHE TROVATO CON ISBN == None

#Adesso droppiamo gli attributi inutili e rinominiamo quelli utili

DFmerged_almost_definitive = DFmerged_isbn.drop("total_review").drop("total_votes").drop("average_rating").drop("isbn").withColumnRenamed("total_count", "total_votes_or_loans").withColumnRenamed("new_average_rating", "average_rating").withColumnRenamed("new_isbn", "isbn").withColumnRenamed("new_total_review", "total_review").withColumnRenamed("new_author", "author")

#DFmerged_almost_definitive.orderBy(rand()).show(20, False)



In [ ]:
#Adesso è il momento di attaccare gli indici di riga e colonna per la compressed sparse matrix tramite le funzioni che ho creato qualche cella fa.

#DFmerged_almost_definitive.filter(typeof(DFmerged_almost_definitive.person_id) != str).show()
RDDmerged_almost_definitive = DFmerged_almost_definitive.rdd
#from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))
book_dictionary = create_book_dictionary(RDDmerged_almost_definitive)
user_dictionary = create_user_dictionary(RDDmerged_almost_definitive)
#n_items = take(20, user_dictionary.items())

#print(n_items)
def addRowColumnId(line): #Funzione usata per aggiungere gli indici per ogni libro e utente
    book_number = book_dictionary[line.book_id] 
    user_number = user_dictionary[line.person_id]
    
    return Row(line.person_id, line.title, str(line.sub_title), line.item_review, line.data_type, line.book_id, line.author, line.total_votes_or_loans, line.average_rating, line.total_review, line.isbn, str(user_number), str(book_number))
RDDdefinitive = RDDmerged_almost_definitive.map(addRowColumnId)

DFdefinitive = RDDdefinitive.toDF(["person_id", "title", "sub_title", "item_review", "data_type", "book_id", "author", "total_votes_or_loans", "average_rating", "total_review", "isbn", "user_index", "book_index"])
DFdefinitive.show(50, False)